In [2]:
using LinearAlgebra

function DMD_SVD(Y, r, Δt)
    
    X₋ = Y[:,1:end-1]
    X₊ = Y[:,2:end]
    U, Σ, V = svd(X₋)
    
    Uʳ = U[:, 1:r] #12 x 4
    Σʳ = diagm(Σ[1:r])
    Vʳ = V[:, 1:r]
    Ã = Uʳ' * X₊ * Vʳ / Σʳ
    Λ, W = eigen(Ã)
    Φ = X₊ * Vʳ / Σʳ * W
    Ω = log.(Λ)/Δt
    x₁ = X₋[:,1]
    b₁ = Φ \ x₁
    
    return Φ, Ω, b₁, Λ

end  

DMD_SVD (generic function with 1 method)

In [3]:
Y = [1 0 0 1
     0 -1 -2 -3
     1 2 3 4]

DMD_SVD(Y, 3, 0.1)

(ComplexF64[0.9999886975134747 + 0.0im 1.000005651022832 - 1.2386892318171243e-5im 1.000005651022832 + 1.2386892318171243e-5im; 1.1302103302995858e-5 + 0.0im -5.651026205949483e-6 + 9.788419945944438e-6im -5.651026205949483e-6 - 9.788419945944438e-6im; -1.1301975561650579e-5 + 0.0im 5.6509623309608514e-6 - 9.788530575345874e-6im 5.6509623309608514e-6 + 9.788530575345874e-6im], ComplexF64[-0.00011302422657463921 + 0.0im, 5.651211330760731e-5 - 9.788128687957398e-5im, 5.651211330760731e-5 + 9.788128687957398e-5im], ComplexF64[2.609451112412929e9 - 0.01404205934495216im, -1.3047314270521936e9 + 2.2597328911704416e9im, -1.3047314270765126e9 - 2.2597328911563997e9im], ComplexF64[0.9999886976412147 + 0.0im, 1.0000056511793949 - 9.78818400274115e-6im, 1.0000056511793949 + 9.78818400274115e-6im])

In [6]:
function hankel(y)
    m, time_duration = size(y) # m - dimention of output vector y
    q = Int(round(time_duration/2)) # q - is the size of Hankel matrix 
    H = zeros(eltype(y), q * m , q) 
    for r = 1:q, c = 1:q # r - rows, c -columns
        H[(r-1)*m+1:r*m, c] = y[:, r+c-1]
    end
    return H, m
end

function ERA(Y, r) 
    H, m = hankel(Y) # Hankel matrix H, and dimention of output m
    U, Σ, V⁺ = svd(H) # Singular value decomposition of H to U,  Σ,  V†
    s = Diagonal(sqrt.(Σ)) # Matrix square root 
    U = U * s
    V⁺ = s * V⁺
    C = U[1:m, 1:r] # m - dimention of output, r - rank of the system
    U₊ = U[1:end-m, :] # U↑
    U₋ = U[m+1:end, :] # U↓
    A = pinv(U₊) * U₋
    A = A[1:r, 1:r] # r - estimated rank of the system
    x₀ = pinv(U) * H[:,1]
    x₀ = x₀[1:r, 1]   
    return A, C, x₀, Σ
end

ERA (generic function with 1 method)

In [7]:
ERA(Y, 2)

([1.600022253425399 0.11683844876067329; -0.5138216319281339 0.06664441324126717], [-0.23015834734473256 -0.8429679656134962; 0.38926018914888244 -0.4984226983354496; -1.0086787256424976 0.1538774310574034], [-1.1254887259245459, -0.8789887695753913], [4.890062597811334, 1.0427309286132604])

In [13]:
U, Σ, V† = svd(Y)

ErrorException: syntax: invalid character "†" near column 8

In [11]:
V⁺

4×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.143737  -0.886816   0.378018
 0.329841   0.159203   0.644872
 0.536403   0.395019   0.325932
 0.763424  -0.179367  -0.578802

In [12]:
V†

ErrorException: syntax: invalid character "†" near column 2